In [1]:
import numpy as np
import pandas as pd

# install pandas_ta - https://github.com/twopirllc/pandas-ta
import pandas_ta as ta

import matplotlib.pyplot as plt

# Calculate Technical Indicators

In [2]:
btc_price_data_1_year = pd.read_csv("data/bitcoin_historical_data_1_year.csv")
btc_price_data_1_year

,timestamp,open,high,low,close,volume,date,time
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00
...,...,...,...,...,...,...,...,...
528627,2024-10-31 23:56:00,70238.76,70248.97,70248.97,70238.76,1.189134,2024-10-31,23:56:00
528628,2024-10-31 23:57:00,70218.00,70250.00,70238.77,70233.24,4.767082,2024-10-31,23:57:00
528629,2024-10-31 23:58:00,70193.97,70242.25,70232.55,70207.78,9.589688,2024-10-31,23:58:00
528630,2024-10-31 23:59:00,70175.16,70207.79,70207.79,70197.83,7.112237,2024-10-31,23:59:00


### 1. Relative Strength Index (RSI)

**RSI** is a momentum oscillator that measures the speed and change of recent price movements. It is used to identify overbought or oversold conditions in a stock's price, generally over a 14-day period.

- **Formula**: The RSI is calculated as:
  
  $$\text{RSI} = 100 - \frac{100}{1 + RS}$$
  
  where $RS$ (Relative Strength) is the ratio of **average gains** to **average losses** over the lookback period (e.g., 14 days).

- **Interpretation**:
  - **Overbought Condition**: When RSI is above 70, the asset is often considered overbought, suggesting a potential for a pullback.
  - **Oversold Condition**: When RSI is below 30, the asset is considered oversold, suggesting a potential for a rebound.

- **Calculation**:
  - Calculate the **change** in price from one day to the next.
  - Separate the changes into **gains** (positive changes) and **losses** (negative changes).
  - Compute the **average gain** and **average loss** over the 14-day period.
  - Calculate \( RS \) as the ratio of average gain to average loss.
  - Use the RSI formula to convert \( RS \) into an index between 0 and 100.

In [3]:
def calculate_rsi(data, window=14):
    """
    RSI is a momentum oscillator that measures the speed and change of price movements, typically over a 14-period interval.
    Assuming 'data' is a DataFrame with a 'close' price column

    Parameters
    ----------
    data:   a DataFrame with the time series data. A column with the name 'close' must be present in the DataFrame! 
            This column is used to calculate the 'RSI' value.
    window: the time period that is taken into account when calculating the 'RSI'
    """
    delta = data['close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    
    return data

### 2. Moving Average Convergence Divergence (MACD)

**MACD** is a trend-following momentum indicator that shows the relationship between two moving averages of an asset’s price.

- **Formula**:
  - **MACD Line**: $\text{MACD} = \text{EMA}_{\text{short}} - \text{EMA}_{\text{long}}$
  - **Signal Line**: A **9-day EMA** of the MACD line.
  - **MACD Histogram**: The difference between the MACD line and the Signal Line.

  Here, EMA stands for Exponential Moving Average, which gives more weight to recent prices.

- **Common Parameters**:
  - **Short EMA**: Often set to a 12-day EMA.
  - **Long EMA**: Often set to a 26-day EMA.
  - **Signal Line EMA**: Often set to a 9-day EMA of the MACD line.

- **Interpretation**:
  - **MACD Line Crosses Above Signal Line**: This is a bullish signal, indicating a potential buy.
  - **MACD Line Crosses Below Signal Line**: This is a bearish signal, indicating a potential sell.
  - **MACD Divergence**: If the price and MACD are moving in opposite directions, it may signal a reversal.
  - **Histogram**: The MACD histogram shows the distance between the MACD line and the Signal Line. When the histogram grows larger, it indicates a strengthening trend in that direction.

- **Application**:
  - The MACD helps traders see changes in momentum, trend direction, and possible reversal points by analyzing the difference between the short and long EMAs.

In [4]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """
    MACD is calculated using two exponential moving averages (EMA): the 12-day EMA and the 26-day EMA,
    with a 9-day EMA as the signal line.
    
    """
    data['EMA12'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA26'] = data['close'].ewm(span=long_window, adjust=False).mean()
    
    # MACD Line
    data['MACD'] = data['EMA12'] - data['EMA26']
    
    # Signal Line
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    
    return data

### 3. Moving Averages (SMA and EMA)

Moving averages smooth out price data to help identify trends over specific time frames. They are often used to see the underlying trend of an asset’s price and are among the most widely used technical indicators.

#### 3.1. Simple Moving Average (SMA)

- **Definition**: The **SMA** is the average of the closing prices over a specific period. For example, a 10-day SMA is the average closing price over the last 10 days.
  
- **Formula**:
  $$\text{SMA} = \frac{\sum_{i=1}^{N} \text{Price}_i}{N}$$
  where $N$ is the period (e.g., 10 days).

- **Interpretation**:
  - **Trend Identification**: When prices are above the SMA, it suggests an upward trend; when below, it suggests a downward trend.
  - **Crossovers**: When a short-term SMA crosses above a long-term SMA (e.g., 10-day SMA crosses above the 50-day SMA), it generates a bullish signal. The reverse crossover indicates a bearish signal.

#### 3.2. Exponential Moving Average (EMA)

- **Definition**: The **EMA** is a weighted moving average that gives more importance to recent prices, making it more responsive to new information than the SMA.
  
- **Formula**:
  - EMA uses a multiplier:
    $$\text{EMA}_\text{current} = \left(\frac{2}{N+1}\right) \times (\text{Price}_\text{current} - \text{EMA}_\text{previous}) + \text{EMA}_\text{previous}$$
    where $N$ is the number of periods.

- **Interpretation**:
  - **More Sensitive to Price Changes**: Because the EMA responds more quickly to recent prices, it is useful in identifying potential reversals and shorter-term trends.

In [5]:
def calculate_moving_averages(data, sma_window=20, ema_window=20):
    """
    Simple Moving Average (SMA) is the average price over a specified number of periods, 
    while Exponential Moving Average (EMA) gives more weight to recent prices.
    
    """
    # Simple Moving Average
    data['SMA'] = data['close'].rolling(window=sma_window).mean()
    
    # Exponential Moving Average
    data['EMA'] = data['close'].ewm(span=ema_window, adjust=False).mean()
    
    return data

In [6]:
# Calculate RSI
btc_price_data_1_year = calculate_rsi(btc_price_data_1_year)
print(btc_price_data_1_year[['close', 'RSI']])

# Calculate MACD
btc_price_data_1_year = calculate_macd(btc_price_data_1_year)
print(btc_price_data_1_year[['close', 'MACD', 'Signal_Line']])

# Calculate moving averages
btc_price_data_1_year = calculate_moving_averages(btc_price_data_1_year)
print(btc_price_data_1_year[['close', 'SMA', 'EMA']])

           close        RSI
0       34667.88        NaN
1       34642.82   0.000000
2       34656.56  35.412371
3       34629.34  20.811875
4       34622.27  18.798741
...          ...        ...
528627  70238.76  20.028025
528628  70233.24  14.761536
528629  70207.78  11.458672
528630  70197.83  11.677624
528631  70209.92  18.516840

[528632 rows x 2 columns]
           close       MACD  Signal_Line
0       34667.88   0.000000     0.000000
1       34642.82  -1.999088    -0.399818
2       34656.56  -2.446476    -0.809149
3       34629.34  -4.940509    -1.635421
4       34622.27  -7.402210    -2.788779
...          ...        ...          ...
528627  70238.76 -50.163495   -38.437037
528628  70233.24 -52.136426   -41.176915
528629  70207.78 -55.119020   -43.965336
528630  70197.83 -57.621405   -46.696549
528631  70209.92 -57.960865   -48.949412

[528632 rows x 3 columns]
           close         SMA           EMA
0       34667.88         NaN  34667.880000
1       34642.82         NaN  34

In [7]:
btc_price_data_1_year.head(25)

,timestamp,open,high,low,close,volume,date,time,RSI,EMA12,EMA26,MACD,Signal_Line,SMA,EMA
0,2023-11-01 00:00:00,34618.86,34676.51,34656.38,34667.88,48.953000,2023-11-01,00:00:00,NaN,34667.880000,34667.880000,0.000000,0.000000,NaN,34667.880000
1,2023-11-01 00:01:00,34642.54,34687.53,34673.30,34642.82,16.178075,2023-11-01,00:01:00,0.000000,34664.024615,34666.023704,-1.999088,-0.399818,NaN,34665.493333
2,2023-11-01 00:02:00,34637.97,34656.82,34642.53,34656.56,8.753120,2023-11-01,00:02:00,35.412371,34662.876213,34665.322689,-2.446476,-0.809149,NaN,34664.642540
3,2023-11-01 00:03:00,34617.22,34656.56,34656.56,34629.34,11.308610,2023-11-01,00:03:00,20.811875,34657.716796,34662.657304,-4.940509,-1.635421,NaN,34661.280393
4,2023-11-01 00:04:00,34597.99,34630.42,34629.41,34622.27,8.583808,2023-11-01,00:04:00,18.798741,34652.263442,34659.665652,-7.402210,-2.788779,NaN,34657.565118
5,2023-11-01 00:05:00,34601.12,34629.21,34620.51,34606.55,4.765130,2023-11-01,00:05:00,15.471231,34645.230605,34655.731159,-10.500554,-4.331134,NaN,34652.706535
6,2023-11-01 00:06:00,34601.14,34630.82,34606.55,34622.72,3.357925,2023-11-01,00:06:00,28.491141,34641.767435,34653.285888,-11.518453,-5.768598,NaN,34649.850674
7,2023-11-01 00:07:00,34624.88,34678.59,34627.35,34665.79,13.859914,2023-11-01,00:07:00,49.294157,34645.463214,34654.212119,-8.748904,-6.364659,NaN,34651.368705
8,2023-11-01 00:08:00,34637.70,34670.23,34660.89,34644.30,7.768562,2023-11-01,00:08:00,43.045889,34645.284258,34653.477888,-8.193629,-6.730453,NaN,34650.695495
9,2023-11-01 00:09:00,34612.67,34665.75,34644.29,34619.47,4.488087,2023-11-01,00:09:00,37.546947,34641.312834,34650.958785,-9.645951,-7.313553,NaN,34647.721639


Calculating technical indicators like RSI (Relative Strength Index), MACD (Moving Average Convergence Divergence), and Moving Averages can be done with the help of Python's `pandas` library. Additionally, for convenience, the `ta-lib` library or the `pandas_ta` package provides ready-made functions for technical indicators.

Here are methods for calculating each of these indicators manually with `pandas`.

### 4.1. Relative Strength Index (RSI)

RSI is a momentum oscillator that measures the speed and change of price movements, typically over a 14-period interval.

```python
import pandas as pd

def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

# Assuming 'data' is a DataFrame with a 'Close' price column
data = pd.DataFrame({'Close': [...]})  # Insert your close price data here
data = calculate_rsi(data)
print(data[['Close', 'RSI']])
```

### 4.2. Moving Average Convergence Divergence (MACD)

MACD is calculated using two exponential moving averages (EMA): the 12-day EMA and the 26-day EMA, with a 9-day EMA as the signal line.

```python
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    data['EMA12'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['EMA26'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    
    # MACD Line
    data['MACD'] = data['EMA12'] - data['EMA26']
    
    # Signal Line
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    
    return data

# Assuming 'data' is a DataFrame with a 'Close' price column
data = calculate_macd(data)
print(data[['Close', 'MACD', 'Signal_Line']])
```

### 4.3. Moving Averages (Simple Moving Average and Exponential Moving Average)

**Simple Moving Average (SMA)** is the average price over a specified number of periods, while **Exponential Moving Average (EMA)** gives more weight to recent prices.

```python
def calculate_moving_averages(data, sma_window=20, ema_window=20):
    # Simple Moving Average
    data['SMA'] = data['Close'].rolling(window=sma_window).mean()
    
    # Exponential Moving Average
    data['EMA'] = data['Close'].ewm(span=ema_window, adjust=False).mean()
    
    return data

# Assuming 'data' is a DataFrame with a 'Close' price column
data = calculate_moving_averages(data)
print(data[['Close', 'SMA', 'EMA']])
```

### 5.1. Using `pandas_ta` or `ta-lib` (Optional)

If you prefer using pre-built functions, you can install and use libraries like `pandas_ta` or `TA-Lib`. Here's an example with `pandas_ta`:

```python
# Install pandas_ta if not already installed
# !pip install pandas_ta

import pandas as pd
import pandas_ta as ta

# Assuming 'data' is a DataFrame with a 'Close' price column
data['RSI'] = ta.rsi(data['Close'], length=14)
data['MACD'], data['Signal_Line'], _ = ta.macd(data['Close'], fast=12, slow=26, signal=9)
data['SMA'] = ta.sma(data['Close'], length=20)
data['EMA'] = ta.ema(data['Close'], length=20)

print(data[['Close', 'RSI', 'MACD', 'Signal_Line', 'SMA', 'EMA']])
```

Using these methods, you can compute RSI, MACD, and moving averages on stock price data effectively in Python. Let me know if you need further customization or explanations for each indicator!

## Using pandas_ta or ta-lib (Optional)

In [8]:
btc_price_data_1_year_copy = btc_price_data_1_year.copy(deep = True)

In [9]:
btc_price_data_1_year_copy['RSI'] = ta.rsi(btc_price_data_1_year_copy['close'], length=14)
btc_price_data_1_year_copy['MACD'], btc_price_data_1_year_copy['Signal_Line'], _ = ta.macd(btc_price_data_1_year_copy['close'], fast=12, slow=26, signal=9)
btc_price_data_1_year_copy['SMA'] = ta.sma(btc_price_data_1_year_copy['close'], length=20)
btc_price_data_1_year_copy['EMA'] = ta.ema(btc_price_data_1_year_copy['close'], length=20)

btc_price_data_1_year_copy[['close', 'RSI', 'MACD', 'Signal_Line', 'SMA', 'EMA']]

,close,RSI,MACD,Signal_Line,SMA,EMA
0,34667.88,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
1,34642.82,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
2,34656.56,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
3,34629.34,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
4,34622.27,NaN,MACD_12_26_9,MACDh_12_26_9,NaN,NaN
...,...,...,...,...,...,...
528627,70238.76,26.231313,MACD_12_26_9,MACDh_12_26_9,70352.5490,70338.173592
528628,70233.24,25.661202,MACD_12_26_9,MACDh_12_26_9,70338.1685,70328.179916
528629,70207.78,23.160865,MACD_12_26_9,MACDh_12_26_9,70325.1425,70316.713258
528630,70197.83,22.248494,MACD_12_26_9,MACDh_12_26_9,70313.8910,70305.391043
